# Flujo completo RAG


In [20]:
import os
from typing import List
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
import re
from typing import List, Dict
from langchain.document_loaders import PyPDFLoader

from dotenv import load_dotenv
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import RetrievalMode
from langchain_huggingface import HuggingFaceEmbeddings


import os
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Script Limpieza y Segmentación

In [21]:
# Función para cargar documentos PDF 
def load_pdf_documents(file_path: str) -> List[str]:
    """
    Carga documentos PDF y devuelve una lista de páginas como texto.

    Args:
        file_path (str): Ruta del archivo PDF a cargar.

    Returns:
        List[str]: Lista de textos extraídos de cada página del PDF.
    """
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    return [doc.page_content for doc in docs]  # Extrae texto como lista de strings

def load_pdf_all_documents(directory_path: str) -> List[str]:
    """
    Carga documentos PDF desde una carpeta y devuelve una lista de páginas como texto.

    Args:
        directory_path (str): Ruta de la carpeta que contiene los archivos PDF.

    Returns:
        List[str]: Lista de textos extraídos de cada página de todos los PDFs en la carpeta.
    """
    all_texts = []
    for filename in os.listdir(directory_path):
        if filename.lower().endswith('.pdf'):  # Filtrar solo archivos PDF.
            file_path = os.path.join(directory_path, filename)
            print(f"Cargando archivo: {file_path}")
            reader = PdfReader(file_path)
            for page in reader.pages:
                all_texts.append(page.extract_text())  # Agregar texto de cada página a la lista
    return all_texts

# Función para limpiar el texto y excluir secciones específicas
def clean_text_and_exclude_sections(text: str) -> str:
    """
    Limpia el texto extraído de un PDF, excluyendo índices y bibliografías 
    basados en patrones comunes.

    Args:
        text (str): Texto extraído del PDF.

    Returns:
        str: Texto limpio y sin secciones de índice o bibliografía.
    """
    # Reemplazar saltos de línea por espacios
    text = re.sub(r'\n+', ' ', text)
    
    # Eliminar espacios múltiples
    text = re.sub(r'\s+', ' ', text)
    
    # Excluir secciones específicas (Índice, Referencias, Bibliografía)
    text = re.sub(r'(?i)(Índice|Table of Contents).*?(Referencias|Bibliografía|References)', '', text, flags=re.DOTALL)
    
    # Opcional: Detectar bibliografías al final del documento
    text = re.sub(r'(?i)(Referencias|Bibliografía|References).*$', '', text, flags=re.DOTALL)
    
    # Eliminar caracteres no deseados (mantener alfanuméricos y signos de puntuación básicos)
    text = re.sub(r'[^a-zA-Z0-9.,;?!:()\s]', '', text)
    
    # Corrección de palabras divididas por guiones al final de línea (ej., "pro-\nject" -> "project")
    text = re.sub(r'-\s', '', text)
    
    # Eliminar espacios al inicio y final
    text = text.strip()
    
    return text


# Función para dividir texto en oraciones
def split_text_into_sentences(text: str) -> List[Dict[str, str]]:
    """
    Divide un texto en oraciones basado en '.', '?', y '!' y devuelve una lista de diccionarios.

    Args:
        text (str): El texto a dividir.

    Returns:
        List[Dict[str, str]]: Lista de diccionarios con 'sentence' y 'index'.
    """
    single_sentences_list = re.split(r'(?<=[.?!])\s+', text.strip())
    sentences = [{'sentence': sentence, 'index': i} for i, sentence in enumerate(single_sentences_list)]
    return sentences


# Función para combinar oraciones
def combine_sentences(sentences: List[Dict[str, str]], buffer_size: int = 1) -> List[Dict[str, str]]:
    """
    Combina oraciones de acuerdo al tamaño del buffer definido.

    Args:
        sentences (List[Dict[str, str]]): Lista de oraciones con índices.
        buffer_size (int): Número de oraciones antes y después a combinar.

    Returns:
        List[Dict[str, str]]: Lista con oraciones combinadas.
    """
    for i in range(len(sentences)):
        combined_sentence = ''

        # Añadir oraciones previas
        for j in range(i - buffer_size, i):
            if j >= 0:
                combined_sentence += sentences[j]['sentence'] + ' '

        # Añadir oración actual
        combined_sentence += sentences[i]['sentence']

        # Añadir oraciones posteriores
        for j in range(i + 1, i + 1 + buffer_size):
            if j < len(sentences):
                combined_sentence += ' ' + sentences[j]['sentence']

        # Guardar la oración combinada en el dict actual
        sentences[i]['combined_sentence'] = combined_sentence.strip()

    return sentences


# Embeddings 

def create_qdrant_store(model_name, combined_sentences):
    """
    Crea y devuelve un QdrantVectorStore a partir de un modelo de embeddings y una lista de frases combinadas.
    
    :param model_name: Nombre del modelo de embeddings (str).
    :param combined_sentences: Lista de diccionarios con las claves 'combined_sentence' y 'index' (list).
    :return: Objeto QdrantVectorStore.
    """
    # Crear embeddings con el modelo especificado
    open_source_embeddings = HuggingFaceEmbeddings(model_name=model_name)

    # Preparar documentos para Qdrant
    documents_for_qdrant = [
        Document(page_content=doc["combined_sentence"], metadata={"index": doc["index"]})
        for doc in combined_sentences
    ]

    # Crear la tienda de vectores en memoria
    qdrant = QdrantVectorStore.from_documents(
        documents_for_qdrant,
        embedding=open_source_embeddings,
        location=":memory:",  # Puedes cambiar la ubicación si necesitas persistencia
        collection_name="my_documents",
        retrieval_mode=RetrievalMode.DENSE,
    )
    
    return qdrant




def create_rag_chain(model, openai_api_key, qdrant):
    """
    Crea una cadena RAG (Retrieval-Augmented Generation) usando LangChain.
    
    :param model: Nombre del modelo OpenAI (str).
    :param openai_api_key: Clave de la API de OpenAI (str).
    :param qdrant: Objeto QdrantVectorStore configurado como un retriever.
    :return: Objeto rag_chain.
    """
    # Configurar el modelo OpenAI
    llm = ChatOpenAI(
        model=model,
        temperature=0.7,  # Ajusta la creatividad según sea necesario
        openai_api_key=openai_api_key
    )

    # Descargar y configurar el prompt desde LangChain Hub
    prompt = hub.pull("rlm/rag-prompt")

    # Función para formatear los documentos
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Configurar el retriever desde Qdrant
    retriever = qdrant.as_retriever()

    # Crear la cadena RAG
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    
    return rag_chain


In [24]:
#Parametros
model = "gpt-3.5-turbo"
openai_api_key = os.getenv("OPENAI_API_KEY")
model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
directory_path = "../practicos-rag/data/USA/"
buffer_size = 2  # Número de oraciones antes y después a combina

## Funciones
# Cargar texto del PDF
pdf_texts = load_pdf_all_documents(directory_path)
# Combinar texto de todas las páginas
full_text = " ".join(pdf_texts)

# Limpiar texto
cleaned_text = clean_text_and_exclude_sections(full_text)

# Dividir en oraciones
sentences = split_text_into_sentences(cleaned_text)

# Combinar oraciones
combined_sentences = combine_sentences(sentences, buffer_size)
# Crear el Qdrant store
qdrant_store = create_qdrant_store(model_name, combined_sentences)

# Crear la cadena RAG
rag_chain = create_rag_chain(model, openai_api_key, qdrant_store)

#Inferencia 
rag_chain.invoke("What are the mandatory data elements that must be submitted in the Automated Commercial Environment (ACE) for articles regulated by the FDA?")

Cargando archivo: ../practicos-rag/data/USA/CFR-2024-vol8.pdf
Cargando archivo: ../practicos-rag/data/USA/CFR-2024-vol3.pdf
Cargando archivo: ../practicos-rag/data/USA/CFR-2024-vol2.pdf
Cargando archivo: ../practicos-rag/data/USA/CFR-2024-vol1.pdf


'The mandatory data elements that must be submitted in the Automated Commercial Environment (ACE) for articles regulated by the FDA include Pasteurization, method of bacterial count, authority to sample and inspect, and scoring. These elements are essential for compliance with regulations under the Federal Import Milk Act.'